In [1]:
import sys
sys.path.append("..")  # points to project root
from src.data import load_data
from src.data import build_datasets

filename = "../data/training_data.csv"

df = load_data(filename)

X = df.text
y = df.labels

X_train, X_test, y_train, y_test = build_datasets(X, y)

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.to(device)


Loading weights: 100%|██████████| 202/202 [00:00<00:00, 2608.93it/s, Materializing param=pooler.dense.weight]                                       
DebertaV2ForSequenceClassification LOAD REPORT from: MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
Key                             | Status     |  | 
--------------------------------+------------+--+-
deberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [30]:

sample_text = X_train[0]
sample_answer = y_train[0]

classes = ["fake news", "real news"]
hypothesis_template = "This news is {}."

results = []

for label in classes:
    hypothesis = hypothesis_template.format(label)
    input = tokenizer(sample_text, hypothesis, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model(**input)
    
    probs = torch.softmax(output["logits"], dim=-1)
    
    entailment_prob = probs[0][0].item()
    results.append(entailment_prob)


final_probs = torch.softmax(torch.tensor(results), dim=-1).tolist()
predictions = dict(zip(classes, final_probs))
print(predictions)



{'fake news': 0.4997272491455078, 'real news': 0.5002727508544922}


In [23]:
tags = {"fake_news":0, "real_news":1}

print(sample_text)
sample_answer

donald trump sends out embarrassing new year‚s eve message; this is disturbing


0

In [31]:
# Returns 'cuda:0' if on GPU, 'cpu' if on CPU
print(next(model.parameters()).device) 

# Boolean check
print(next(model.parameters()).is_cuda) 


cuda:0
True
